<div class="text-muted">

# Lecture Notes

Instructions:

> **Week k (week that group topics are presented in lecture)** <br>
By Friday at 4pm, the group is required to submit their typed-up notes from that week’s
lecture. This will form the basic outline of their deliverable and they will receive
feedback from the TA on their work. <br><br>
The group should also have an outline of the one outside paper that they intend to
describe in greater detail.

## L13. Constraint Programming
- Big picture
- Viewing Sodoku and n-Queens as CSPs
- Constraint Modeling
- Applications of CSPs
- Tying to other course content

## L14. Solving Constraint Programs - Propagation and Basic Search
- Constraint propagation: Achieving arc consistency with AC-1 and AC-3
- Examples on integer domains with algebra
- Complexity of AC-1 and AC-3
- Arc consistency: sound but not complete

## L15. Solving Constraint Programs - Backtrack Search and Forward Checking
- Narrative: show the largest reasonable n-Queens problem that can be solved for various solution methods
- Combining inference with search
    - Why standard search sucks
    - Backtrack search
- Constraint Optimization Problem: Branch & Bound
- Forward Checking: Combining backtracking with **limited** constraint propagation
- Dynamic variable ordering and BT-FC
- "Bucket Elimination", a form of adaptive consistency (i.e., variable elimination)
    - Viewing arc consistency as variable elimination

###### Optional Content, Not Discussed in Lecture
- Adaptive consistency is linear for trees
- Iterative repair
- Conflict-directed back jumping

</div>

---

# Constraint Programming
Topics: constraint propagation, search, variable elimination; global constraints; MVRP

## 1. Introduction

Constraints naturally arise in a variety of interactions and fields of study such as game theory, social studies, operations research, engineering, and artificial intelligence. A constraint refers to the relationship between the state of objects, such as the constraint that the three angles of a triangle must sum to 180 degrees. Note that this constraint has not precisely stated each angle's value and still allows some flexibility. Said another way, the triangle constraint restricts the values that the three variables (each angle) can take, thus providing information that will be useful in finding values for the three angles.

Another example of a constrained problem comes from the recently-aired hit TV series *Buddies*, where a group of five (mostly mutual) friends would like to sit at a table with three chairs in specific arrangements at different times, but have requirements as to who they will and will not sit with.

Another example comes from scheduling: at the university level, there is a large number of classes that must be scheduled in various classrooms such that no professor or classroom is double booked. Further, there are some constraints on which classes can be scheduled for the same time, as some students will need to be registered for both.

Computers can be employed to solve these types of problems, but in general these tasks are computationally intractible and cannot be solved efficiently in all cases with a single algorithm [1]. However, by formalizing these types of problems in a constraint processing framework, we can identify classes of problems that can be solved using efficient algorithms.

### 1.1. Modelling

A **constraint satisfaction problem** (CSP) is formalized by a *constraint network*, which is the triple $\langle V,D,C\rangle$, where
- $V = \{V_i\}_{i=1}^n$ is the set of $n$ variables
- $D = \{D_i\}_{i=1}^n$ is the set of variable domains, where the domain of variable $V_k$ is $D_k$
- $C = \{C_i\}_{i=1}^m$ is the set of constraints on the values that each $V_i$ can take on. Specifically,
    - Each constraint $C_i = \langle S_i,R_i\rangle$ specifies allowed variable assignments.
    - $S_i \subset V$ contains the variables involved in the constraint, called the *scope* of the constraint.
    - $R_i$ is the constraint's *relation* and represents the simultaneous legal value assignments of variables in the associated scope.
        - For example, if scope of the first constraint is $S_1 = \{V_3, V_8\}$, then the relation $R_1$ is a subset of the Cartesian product of those variables' domains: $R_1 \subset D_3 \times D_8$, and an element of the relation $R_1$ could be written as a 2-tuple $(x,y)\in R_1$.

Note that for a CSP model, *any* consistent assignment of the variables (i.e., where all constraints are satisfied) constitutes a valid solution, it may not be the "best" solution. Notions of optimality can be captured by introducing an objective function which is used to find a valid solution with the lowest cost. This is referred to as a **constraint *optimization* problem** (COP). We will refer generally to CSPs with the understanding that a CSP can easily become a COP.

### 1.1.1. Modelling as a Graph
<div class="alert alert-danger">
<b>TODO:</b> Discuss the graphical view of constraints. Let's also assume that we are only considering binary CSPs and note that any n-connected CSP can be reformulated into an equivalent binary CSP.
</div>

### 1.2. Solving

The goal of formalizing a CSP as a constraint network model is to efficiently solve it using computational algorithms and tools. **Constraint programming** (CP) is a powerful tool to solve combinatorial constraint problems and is the study of computational systems based on constraints. Once the problem has been modeled as a formal CSP, a variety of computable algorithms could be used to find a solution that satisfies all constraints. We will restrict ourselves to using CP to solve CSPs with **discrete, finite domains**.

In general, there are two methods used to solve a CSP: search or inference. In previous 16.410/413 problems, **state-space search** was used to find the best path through some sort of graph or tree structure. Likewise, state-space search could be used to find a valid "path" through the CSP that satisfies each of the local constraints and is therefore a valid global solution. However, this approach would quickly become intractable as the number of variables and the size of each of their domains increase.

In light of this, the second solution method becomes more attractive. **Constraint propagation**, a specific type of inference, is used to reduce the number of legal values from a variable's domain by pruning values that would violate the constraints of the given variable. By making a variable locally consistent with its constraints, the domain of adjacent variables may potentially be further reduced as a result of missing values in the pairwise constraint of the two variables. In this way, by making the first variable consistent with its constraints, the constraints of neighboring variables can be re-evaluated, causing a further reduction of domains through the propagation of constraints. These ideas will later be formalized as $k$-consistency.

Constraint propagation may be combined with search, using the pros of both methods simultaneously. Alternatively, constraint propagation may be performed as a pre-processing pruning step so that search has a smaller state space to search over. Sometimes, constraint propagation is all that is required and a solution can be found without a search step at all.

After giving examples of modelling CSPs, this notebook will explore a variety of solution methods based on constraint propagation and search.

---

## 2. Modeling

Given a constrained problem, it is desirable to identify an appropriate constraint network model $\langle V,D,C\rangle$ that can be used to find its solution. Modelling for CSPs is an important step that can dramatically affect the difficulty in enumerating the associated constraints or efficiency of finding a solution.

### 2.1. Examples

In this section, we consider two puzzle problems and model them as CSPs.

<div class="alert alert-danger">
<b>TODO:</b> Make these models more formal?
</div>

### 2.1.1. Sudoku

Sudoku is a well-known puzzle that can be easily solved using the CSP framework. Given a $9\times 9$ grid partially filled with numbers from $1$ to $9$, the goal is to fill the grid entirely such that:

- each $3\times 3$ sub-grid contains all numbers from $1$ to $9$
- each row contains all numbers from $1$ to $9$
- each column contains all numbers from $1$ to $9$

<table><tr>
<td> <img src="images/start_sudoku.png"/> </td>
<td> <img src="images/end_sudoku.png" /> </td>
</tr></table>

### 2.1.2. N-Queens

The N-queens problem is a well-know puzzle among computer scientists. Given any integer $N$, the goal is to place $N$ queens on an $N\times N$ chessboard satisfying the constraint that no two queens threaten each other. A queen can threaten any other queen that is on the same row, column, or diagonal.

<img src="images/nqueens.png" align="center"/>

---

## 3. Constraint Propagation Methods

Give an overview of consistency

### 3.1. Node Consistency

#### 3.1.1. Algorithm: REVISE

- REVISE is a node consistency thing, right?

### 3.2. Arc Consistency

Talk about sound but not complete. Is consistency in general "sound but not complete", or is this just an arc consistency thing?

#### 3.2.1. Algorithm: AC-1

- Give the algorithm
- Discuss complexity

#### 3.2.2. Algorithm: AC-3

- Give the algorithm
- Discuss complexity

### 3.3. Path Consistency (<-- Should we do this?)

### 3.4. $k$-Consistency (<-- Should we do this?)

### 3.5. Example: integer domains with algebra

- Is this where **global constraints** can be discussed? It looks like that's how Russell and Norvig do it.

---

## 4. Search Methods

### 4.1. Standard Search (<-- Should we do this?)

### 4.2. Backtrack (BT) Search

---

## 5. Hybrid Methods

### 5.1. BT Search with Forward Checking (BT-FC)

### 5.2. BT-FC with Dynamic Variable Ordering

### 5.3. Adaptive Consistency: Bucket Elimination

---

# Symmetries

<img src="images/escher_2.jpg" align="center"/>
<div style="text-align: right"> M. C. Escher </div>


## Introduction

A CSP often exhibit some symmetries, which are mappings that preserve satisfiability of the CSP. They are particularly harmful when we are looking for all possible valid assignment of a CSP since symmetries have many solutions that are just a symmetric solution of another one.

> **Definition (Symmetry).** For any CSP instance $P = \langle X, D, C \rangle$, a solution symmetry of $P$ is a permutation of the set $X\times D$ that preserves the set of solutions to $P$.

In other words, a solution symmetry is a bijective mapping defined on the set of possible variable-value pairs of a CSP, that maps solutions to solutions.

## Why is symmetry important? 

A principal reason for identifying symmetry in CSPs is to reduce search effort by not exploring assignments that are symmetrically equivalent to assignments considered elsewhere in the search. Clearly, if the solution symmetry group is larger than the constraint symmetry group, there will potentially be a greater search reduction from using the solution symmetries, if they can somehow be identified in advance.


## Case Study: symmetries in N-Queens problem

The standard formulation of the N-Queens problem as a CSP has $N$ variables corresponding to the rows of the chessboard, say $Q_1,Q_2,\ldots,Q_n$. The domain of values corresponds to the columns of the chessboard, say $D = \{1, 2,\ldots, n\}$. The constraints can be expressed as follows:
- the values of $Q_1, Q_2, \ldots, Q_n$ are all different; 
- forall $i,j,1\leq i< j \leq N,|Q_i−Q_j \neq |i−j|$.

A chessboard has eight geometric symmetries: 
- horizontal reflection $r_h$
- vertical reflection $r_v$
- reflectins along the two diagonal axes ($r_{d_1}$ and $r_{d_2}$)
- rotations through $90$&deg;, $180$&deg; and $270$&deg ($r_{90}$, $r_{180}$, $r_{270}$)
- identity (no-reflections) $r_I$

We can group all the symmetries in $G = \{r_I, r_h,r_v, r_{d_1}, r_{d_2},r_{90}, r_{180}, r_{270} \}$
For the seek of simplicity let's denote $A^g$, with $g\inG$ as the result of the application of symmetry $g$ to $A$. For example, in the 4-Queens problem $(Q_1 = 2)^{r_{90}} = (Q_2=4)$.

An example can be given by the 4-Queens problem, all the solutions are the following

<img src="images/two-queens_symmetries.png" align="center"/>

It's easy to note that the second solution is the first solution when we flip the chessboard horizontally.



## Avoid symmetries

### Adding Constraints Before Search

Symmetry in CSPs is usually identified, in practice, by applying human insight: the programmer sees that some transformation would transform a hypothetical solution into another hypothetical solution. Then the programmer can try to formalize some constraint that preserve solutions but remove some of the symmetries.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of N-Queens problem with broken symmetries through additional constraints
</div>

This approach is fine if done correctly, but can obviously lose solutions if done incorrectly.


### Dynamic Symmetry Breaking Methods

Dynamic symmetry breaking methods are those that operate to break symmetry during the search process, avoiding some brach of the search tree that are symmetrically equivalent to others. Let's explore some of them.

#### Symmetry Breaking During Search (SBDS)

The basic idea of SBDS (this is the name of the algorithm, not a class for them) is to add constraints to a problem so that, after backtracking from a search decision, the SBDS constraints ensure that no symmetric equivalent of that decision is ever allowed. This is a dynamic technique, since we cannot add the constraints until we know what search decision is being made.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example search tree for SBDS
</div>

A big drawback of this agorithm is that it always demand from the constraint programmer to explicitly state all symmetries $g\in G$a. If a problem has a large number of symmetries there may be too many for the user to identify and implement by hand.

### The Lex-Leader Method

Puget proved that whenever a CSP has symmetry [2], it is possible to find a _'reduced form'_, with the symmetries eliminated, by adding constraints to the original problem. Puget found such a reduction for three simple constraint problems, and showed that this reduced CSP could be solved more efficiently than in its original form. From this work a method called **lex-leader** was developed, the background idea is rather simple, for each equivalence class of solutions under our symmetry group, we will predefine one to be the canonical solution. We will achieve this by choosing a static variable ordering. 

For example let's consider a problem where we have three variables $x_1$ $x_2$ and $x_3$ subject to `alldifferent` constraint and domain $\{1,2,3\}$. This problem has $3!$ solutions, the easiest way to impose a canonical solution is to impose that $x_1 \leq x_2 \leq x_3$, that is the _lexicographic_ ordering. Adding this additional constraint we will get exactly one solution. All the other can be recovered applying all the symmetries.

In general the lex-leader method impose a variable ordering for Each permutation in the symmetry group, formally

$$ \forall g \in G, \, A\preceq_{\text{lex }} A^g $$

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of model with/without lex-leader
</div>

An important practical issue with the lex-leader constraints is that they do not “respect” the variable and value ordering heuristics used in search. That is, it may well be that the leftmost solution in the search tree, which would otherwise be found first, is not canonical and so is disallowed, leading to increased search. This is in contrast to techniques such as SBDS, which do respect the heuristic.

#### Double-Lex

A matrix of decision variables has row symmetry iff given a solution, any permutation of the rows is also a solution. Similarly, it has column symmetry iff given a solution, any permutation of the columns is also a solution.  Row and column symmetry occurs in many mod- els with matrices of decision variable. We have seen that that we can add $n!m!$  constraints to break all the symmetries. Adding so many constraints can be counter-productive. A common approach is order the rows of a matrix and **independently** order the columns. This produces only $n + m − 2$ symmetry breaking constraints. This method is called **double-lex** and has the disadvantage of breaking only some of the symmetries.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of the same model as before but with double-lex
</div>

## Intractability of Breaking Symmetry


It is worth to mention that lex-leader requires one constraint for each element of the group. 
In the case of a matrix with $m$ rows and $n$ columns, this is $m!n!$, which is impractical in general. Therefore there are many cases where lex-leader is applicable but impractical.

> **Theorem (Walsh 2011)** Given any _simple_ ordering, there exists a symmetry group such that deciding if an as- signment is smallest in its symmetry class according to this ordering is NP-hard.

Since breaking symmetry appears intractable in gen- eral, a major research direction is to identify special cases which occur in practice where the symmetry group is more tractable to break.

> **Theorem (Katsirelos, Narodytska, and Walsh 2010)** Propagating the double-lex constraint is NP-hard.

# References

- [1] Rina Dechter. 2003. Constraint Processing. Morgan Kaufmann Publishers Inc., San Francisco, CA, USA.
- [2] J.-F. Puget. On the satisfiability of symmetrical constraint satisfaction problems. In Methodologies for Intelligent Systems (Proceedings of ISMIS’93), LNAI 689, pages 350–361. Springer, 1993.